In [1]:
!pip install langchain sentence-transformers faiss-cpu langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 721.8 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 820.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 947.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 761.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.0/273.0 kB 954.8 kB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 760.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 880.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 976.8 kB/s eta 0:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 800.9 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 571.5 kB/s eta 0:00:00a 0:00:01
  Attempting unins

In [2]:
import pandas as pd
import numpy as np
import torch

from langchain_core.documents import Document
from langchain_community.embeddings.openai import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import re
from datetime import datetime
from tqdm import tqdm

import os
import getpass

from sklearn.metrics import f1_score
nltk.download('punkt')
stop_words = stopwords.words("french")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-03-28 07:40:18.685337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 07:40:18.685456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 07:40:18.853666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Import datas

In [4]:
data = pd.read_csv("../datas/psg_date.csv")
train = pd.read_csv("../datas/train_data.csv")

data.rename(columns={"contexte":"texte"}, inplace=True)

data['texte'] = data['texte'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


data["date_accident"] = train["date_accident"]
data["date_consolidation"] = train["date_consolidation"]

# Insert your API key

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI key:")

# Function for preprocess your datas

In [14]:
def preprocess_documents(df_or_series):
    pages_content = []
    
    if isinstance(df_or_series, pd.Series):
        df = df_or_series.to_frame().transpose()
    else:
        df = df_or_series
    
    for index, row in df.iterrows():
        texte = row["texte"]
        name = row["filename"]
        
        # Découpe le texte en phrases
        phrases = sent_tokenize(texte)
        
        for phrase in phrases:
            texte_doc = Document(page_content=phrase,
                                 metadata={'name': name})
            pages_content.append(texte_doc)
        
    return pages_content

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# After that, we define our retriveal function

In [50]:
def retriver(df, query, openai_model, embedding_model_name):
    res_date = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):

        texte = row["texte"]
        filename = row["filename"]

        content = preprocess_documents(pd.Series({"texte": texte, "filename": filename}))

        if openai_model != None:
            model = ChatOpenAI(model=openai_model, temperature=0.2)

        else :
            model = ChatOpenAI(temperature=0.2)

        if embedding_model_name == None:
            embedding_model = OpenAIEmbeddings()
            faiss_index = FAISS.from_documents(content, embedding_model)

        else:
            embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
            faiss_index = FAISS.from_documents(content, embedding_model)
        
        retriever = faiss_index.as_retriever(search_kwargs={'k': 7}, search_type="mmr")

        if query == "what is the date of accident?":
            template = """You are a legal assistant. You will look for information in the following document::
            {context}.
            You will search for one of the following information :
            - the date of the accident : The date of the accident refers to the date on which the accident occurred
            You will only return the date with the following format : YYYY-MM-DD. I don't want text around it
            The user is going to ask you the following thing: {question}
            """

            prompt = ChatPromptTemplate.from_template(template)

            chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | model
            | StrOutputParser()
            )

            val = chain.invoke(query)
            # print(val)
            res_date.append(val)

        elif query == "what is the date of consolidation?":
            template = """You are a legal assistant. You will look for information in the following document::
            {context}.

            You will search for one of the following information :
            - the date of consolidation : The consolidation date is the date on which the victim's injuries
            became stable and were declared definitive by a doctor.
            The information should be present in most cases, but sometimes it is either missing (you will display the value "n.c.") or not applicable (you will display the value "n.a.") if the injury did not stabilize before the victim's death.

            You will return the date with the following format : YYYY-MM-DD. I don't want text around it
            If the date of consolidation is missing in the document, you will return the value n.c.
            If the injury did not stabilize before the victim's death, you will return the value n.a.

            The user is going to ask you the following thing: {question}
            """

            prompt = ChatPromptTemplate.from_template(template)

            chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | model
            | StrOutputParser()
            )

            val = chain.invoke(query)
            res_date.append(val)
    return res_date

# We normalize our outputs

In [ ]:
french_months = {
    'janvier': 1, 'février': 2, 'mars': 3, 'avril': 4, 'mai': 5, 'juin': 6,
    'juillet': 7, 'août': 8, 'septembre': 9, 'octobre': 10, 'novembre': 11, 'décembre': 12
}

def extract_and_reformat_date(text):
    if 'n.c.' in text:
        return "n.c."
    elif "n.a." in text: 
        return 'n.a.'
    
    date_pattern_ymd = r"\d{4}-\d{2}-\d{2}"
    date_pattern_dmy = r"(\d{1,2}),?\s([a-zA-Zéû]+)\s(\d{4})"
    date_pattern_slash = r"(\d{1,2})/(\d{1,2})/(\d{4})"
    date_pattern_mdy = r"([a-zA-Z]+) (\d{1,2}), (\d{4})"
    
    match_ymd = re.search(date_pattern_ymd, text)
    match_dmy = re.search(date_pattern_dmy, text)
    match_slash = re.search(date_pattern_slash, text)
    match_mdy = re.search(date_pattern_mdy, text)
    
    if match_ymd:
        return match_ymd.group(0)
    elif match_dmy:
        day, month_name, year = match_dmy.groups()
        try:
            month_number = datetime.strptime(month_name, "%B").month
        except ValueError:
            month_number = french_months.get(month_name.lower())
            if month_number is None:
                return None
        return f"{year}-{month_number:02d}-{day.zfill(2)}"
    elif match_slash:
        part1, part2, year = match_slash.groups()
        day, month = part1, part2  # Assumant le format JJ/MM/AAAA
        return f"{year}-{int(month):02d}-{int(day):02d}"
    elif match_mdy:
        month_name, day, year = match_mdy.groups()
        try:
            month_number = datetime.strptime(month_name, "%B").month
        except ValueError:
            return None  # En cas de mois inconnu en anglais
        return f"{year}-{month_number:02d}-{day.zfill(2)}"
    else:
        return None

# Now, we apply our RAG

In [ ]:
OPENAI_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL_NAME = "xlm-r-distilroberta-base-paraphrase-v1"

In [ ]:
query = "what is the date of accident?"

res_date_accident = retriver(train, query, OPENAI_MODEL, EMBEDDING_MODEL_NAME)
train["date_accident_pred"] = res_date_accident

In [ ]:
query = "what is the date of consolidation?"

res_date_consolidation = retriver(train, query, OPENAI_MODEL, EMBEDDING_MODEL_NAME)
train["date_consolidation_pred"] = res_date_consolidation

In [ ]:
train['date_accident_pred'] = train['date_accident_pred'].apply(extract_and_reformat_date)
train['date_consolidation_pred'] = train['date_consolidation_pred'].apply(extract_and_reformat_date)
train = train.fillna(value=np.nan)
train = train.fillna(value='n.c.')

# Let's compute our metrics

In [36]:
def compute_accuracy(df, y_true, y_pred):

    good_preds = (df[y_true] == df[y_pred]).sum()
    lenght = df.shape[0]

    accuracy = good_preds / lenght
    return accuracy

accuracy_accident = compute_accuracy(train, 'date_accident', 'date_accident_pred')
accuracy_consolidation = compute_accuracy(train, 'date_consolidation', 'date_consolidation_pred')

print(f"Accuracy accident: {accuracy_accident}")
print(f"Accuracy consolidation: {accuracy_consolidation}")

Accuracy accident: 0.712987012987013
Accuracy consolidation: 0.5467532467532468


In [40]:
def compute_f1(df, y_true, y_pred):

    y_t = (df[y_true] == df[y_true]).astype(int)
    y_p = (df[y_pred] == df[y_true]).astype(int)

    f1 = f1_score(y_t, y_p)

    return f1

f1_accident = compute_f1(train, 'date_accident', 'date_accident_pred')
f1_consolidation = compute_f1(train, 'date_consolidation', 'date_consolidation_pred')

print(f"F1 accident: {f1_accident}")
print(f"F1 consolidation: {f1_consolidation}")

F1 accident: 0.8324488248673237
F1 consolidation: 0.7069689336691856


In [54]:
train.to_csv("../datas/preds_rag_openai.csv")